In [3]:
# 크롤링 한글 
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import bs4
import pandas as pd
import csv

# 크롬 드라이버 설정
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# URL 리스트
file_path = r'.\urls.csv'
essay_urls = []
with open(file_path, "r") as f:
    reader = csv.reader(f)
    for row in reader:
        essay_urls.append(row[0])


def crawling_url(url):
    driver.get(url)
    response = driver.page_source
    one_page = bs4.BeautifulSoup(response, 'html.parser')
    
    # 불필요 요소 제거
    for p_element in one_page.find_all('p', {'class': 'txSpllChk'}):
        p_element.extract()

    for span_element in one_page.find_all('span', {'class': 'sup'}):
        span_element.extract()

    for a_element in one_page.find_all('a'):
        a_element.extract()
        
    # 공백 제거 및 텍스트만 추출
    kwd_list = []
    for text in one_page.find_all('div', {'class': 'tx'}):
        kwd_list.append(text.get_text(separator=' ', strip=True))
    
    return kwd_list

korean_essay_lists = []
for kwd_url in essay_urls:
    kwd_list = crawling_url(kwd_url)
    korean_essay_lists.append(kwd_list)
    time.sleep(5)

# 드라이버 종료
driver.quit()

# 결과 출력
with open('korean1-240.csv', 'a', newline='') as f:
    writer = csv.writer(f)
    for i, kwd_list in enumerate(korean_essay_lists):
        writer.writerow(kwd_list)



In [8]:
# 질문별로 나눠저 있는 답변을 하나의 paragraph화
import csv
essay_list = []
with open('korean1-160.csv', 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        sentence = ' '.join(row)
        essay_list.append(sentence)

In [9]:
# paragraph화해서 파일로 저장
with open('korean1-160_final.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    for i, x in enumerate(essay_list):
        writer.writerow([x])